# About pointers

Many of the problems with pointers come from their multiple uses. They point indifferently to:
* memory areas of the stack or of the heap,
* single objects or tables,
* heap zones whose release is managed elsewhere,
* heap zones to be released at the end of the current instruction block.

**Let's sort it out and express these uses more clearly**.

## Raw pointers and references: `T*` and `T&`

A "raw" pointer (e.g. `T *`) is supposed to have the most ordinary meaning: **it points to an object, but does not "own" it**. In what follows, the portion of code that uses this pointer is not supposed to do a `delete` at the end of the use.

If the pointer itself is not meant to change value, and is never meant to be zero, **we will prefer a reference** whenever possible.

In addition, using a raw pointer to designate an array is to be avoided. Many other solutions exist now.

## Non-zero pointers: `gsl::not_null<T>`

When dereferencing a pointer (via `*` or `->`), a common practice is to first check that it is not null as a precaution, which obscures and slows down the code.

Whenever possible, **replace this pointer with a reference**.

**Otherwise**, use **`gsl::not_null<T>`**, which ensures that you never assign a null value to the pointer.

If you try to assign a null value, you will get an error at compile time (`d2` in the example below), or at run time (`d4` in the example below).

In [1]:
%%file tmp.gsl-pointer.cpp

#include <iostream>
#include <gsl/gsl>

struct Demo {
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
} ;

int main() {
    gsl::not_null<Demo *> d1 { new Demo() } ;
    delete d1 ;
    // ...
    //gsl::not_null<Demo *> d2 { nullptr } ;
    // ...
    Demo * d3 { nullptr } ;
    // ...
    gsl::not_null<Demo *> d4 { d3 } ; 
}

Overwriting tmp.gsl-pointer.cpp


In [2]:
!rm -f tmp.gsl-pointer.exe && g++ -std=c++17 -I./ tmp.gsl-pointer.cpp -o tmp.gsl-pointer.exe

In [3]:
!./tmp.gsl-pointer.exe

Constructor
Destructor
terminate called without an active exception
Aborted (core dumped)


## Ownership pointers `gsl::owner<T>`

The type `gsl::owner<T>` is used to mark that a pointer owns the pointed object. `T` is assumed to be a pointer type, such as `int *`.

The type `gsl::owner<T>` does nothing in itself: you are still in charge of making the call to `delete`, but it clarifies your intention and can allow static checking tools to detect a forgotten delete.

In [4]:
%%file tmp.gsl-pointer.cpp

#include <iostream>
#include <gsl/gsl>

struct Demo {
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
} ;

int main() {
    gsl::owner<Demo *> d { new Demo() } ;
    delete d ; 
}

Overwriting tmp.gsl-pointer.cpp


In [5]:
!rm -f tmp.gsl-pointer.exe && g++ -std=c++17 -I./ tmp.gsl-pointer.cpp -o tmp.gsl-pointer.exe

In [6]:
!./tmp.gsl-pointer.exe

Constructor
Destructor


## Smart pointers `std::unique_ptr<T>` and `std::shared_ptr<T>`

In the case of owning pointers, the GSL encourages the use of pointers from the standard libraries `std::unique_ptr<T>` and `std::shared_ptr<T>`.

`std::unique_ptr<T>`
* costs nothing,
* but is not copiable.

`std::shared_ptr<T>`
* easy to copy,
* but expensive.

Whenever you can, entrust your raw pointer, created by `new`, directly to one of these smart pointers, and only use `gsl::owner<T>` as a last resort. Even better, use `std::make_unique` and `std::make_shared`.

# Take away

#### Non-owning pointers and references, by order of preference

* `T&`: non-owner, cannot be null (always attached to an element).
* `gsl::not_null<T>`: non-owner, T is a pointer, cannot be zero.
* `T*`: non-owner, can be null, assumed to point to a unique element.

#### Owning pointers, by order of preference

* `unique_ptr<T>`: unique owner, not copyable, movable, automatic delete, efficient.
* `shared_ptr<T>`: shared ownership, automatic delete, simple but less efficient.
* `gsl::owner<T>`: owner, T is a pointer, can be zero, assumed to point to a dynamically allocated element (on the heap)`

# Questions ?

# Exercise

In the code below:
* write `my_owner`, which must emulate `gsl::owner` (trivial) ;
* write `my_not_null`, which must emulate `gsl::not_null` (easy) ;
* make sure that your types only accept pointers as parameters (difficult).

In [4]:
%%file tmp.gsl-pointers.h

#include <iostream>

class Demo {
  public:
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    void print() { std::cout<<"Printing"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
} ;

Writing tmp.gsl-pointers.h


In [38]:
%%file tmp.gsl-pointers-owner.cpp

#include "tmp.gsl-pointers.h"
#include <type_traits>

template <typename T, typename = std::enable_if_t<std::is_pointer_v<T>>>
using my_owner = T ;

int main() {
  my_owner<Demo> p1 ;              // COMPILATION ERROR: Demo is not a pointer
  my_owner<Demo *> p2 {new Demo()} ;
  p2->print() ;
  delete p2 ;
}

Overwriting tmp.gsl-pointers-owner.cpp


In [39]:
!rm -f tmp.gsl-pointers-owner.exe && g++ -std=c++17 -I./ tmp.gsl-pointers-owner.cpp -o tmp.gsl-pointers-owner.exe

In file included from /usr/local/include/c++/13.2.0/bits/move.h:37,
                 from /usr/local/include/c++/13.2.0/bits/exception_ptr.h:41,
                 from /usr/local/include/c++/13.2.0/exception:164,
                 from /usr/local/include/c++/13.2.0/ios:41,
                 from /usr/local/include/c++/13.2.0/ostream:40,
                 from /usr/local/include/c++/13.2.0/iostream:41,
                 from tmp.gsl-pointers.h:2,
                 from tmp.gsl-pointers-owner.cpp:2:
/usr/local/include/c++/13.2.0/type_traits: In substitution of ‘template<bool _Cond, class _Tp> using std::enable_if_t = typename std::enable_if::type [with bool _Cond = false; _Tp = void]’:
tmp.gsl-pointers-owner.cpp:9:16:   required from here
/usr/local/include/c++/13.2.0/type_traits:2610:11: error: no type named ‘type’ in ‘struct std::enable_if<false, void>’
 2610 |     using enable_if_t = typename enable_if<_Cond, _Tp>::type;
      |           ^~~~~~~~~~~
tmp.gsl-pointers-owner.cpp: In function 

In [9]:
!./tmp.gsl-pointers-owner.exe

/usr/bin/sh: 1: ./tmp.gsl-pointer.exe: not found


In [34]:
%%file tmp.gsl-pointers-not-null.cpp

#include "tmp.gsl-pointers.h"
#include <type_traits>

template< typename T>
class my_not_null {
  public:
    my_not_null( T ptr ) : m_ptr(ptr) {
        if (!m_ptr) throw "valeur nulle" ;
    }
    my_not_null( std::nullptr_t ) = delete ;
    my_not_null( my_not_null<T> const & other )
     : my_not_null(other.m_ptr) {}
    void operator=( my_not_null<T> const & other ) {
        m_ptr = other.m_ptr ;
        if (!m_ptr) throw "valeur nulle" ;
    }
    T operator->() { return m_ptr; }
    operator T() { return m_ptr; }
  private:
    T m_ptr;
};
    
int main() {
  //my_not_null<Demo*> p1 ;          // COMPILATION ERROR: p1 is not initialized
  //my_not_null<Demo*> p2(nullptr) ; // COMPILATION ERROR: p2 cannot be null
  my_not_null<Demo *> p3 = new Demo() ;
  p3 = nullptr ;                   // EXECUTION ERROR: p3 cannot be null
  p3->print() ;
  delete p3 ;
}

Overwriting tmp.gsl-pointers-not-null.cpp


In [35]:
!rm -f tmp.gsl-pointers-not-null.exe && g++ -std=c++17 -I./ tmp.gsl-pointers-not-null.cpp -o tmp.gsl-pointers-not-null.exe

tmp.gsl-pointers-not-null.cpp: In function ‘int main()’:
tmp.gsl-pointers-not-null.cpp:28:8: error: use of deleted function ‘my_not_null<T>::my_not_null(std::nullptr_t) [with T = Demo*; std::nullptr_t = std::nullptr_t]’
   28 |   p3 = nullptr ;                   // EXECUTION ERROR: p3 cannot be null
      |        ^~~~~~~
tmp.gsl-pointers-not-null.cpp:11:5: note: declared here
   11 |     my_not_null( std::nullptr_t ) = delete ;
      |     ^~~~~~~~~~~


In [25]:
!./tmp.gsl-pointers-not-null.exe

Constructor
Printing
Destructor


## Sources

* http://isocpp.github.io/CppCoreGuidelines/CppCoreGuidelines#gsl-guidelines-support-library
* http://modernescpp.com/index.php/c-core-guideline-the-guidelines-support-library
* http://nullptr.nl/2018/08/refurbish-legacy-code/

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*